# Import Packages

In [1]:
import numpy as np
import cvxpy as cp

# Setting Up

## Functions (definitions)

In [2]:
def initialize_nonogram(N, colors):
    """
    Initializes a nonogram grid with random colors and computes the row and column sums for each color.

    Parameters:
    N (int): The size of the grid (N x N).
    colors (dict): A dictionary of colors and their corresponding labels.

    Returns:
    tuple: A tuple containing the initialized grid and a dictionary with the row and column sums for each color.
    """
    # Initialize the grid with random colors, excluding 'Vacant'
    actual_colors = [color for color in colors if color != 'Vacant']
    grid = np.random.choice(actual_colors, size=(N, N))

    # Compute the colored sums for each row and each column
    color_sums = {color: {'row': np.zeros(N, dtype=int), 'col': np.zeros(N, dtype=int)} for color in actual_colors}
    
    for color in actual_colors:
        # Mask the grid for the current color and calculate the sums
        mask = (grid == color)
        color_sums[color]['row'] = np.sum(mask, axis=1)
        color_sums[color]['col'] = np.sum(mask, axis=0)

    return grid, color_sums

def create_system_matrix(N):
    """
    Create a matrix A for a nonogram of size N that maps from the grid to the row and column sums.
    The top part of A corresponds to the row sums and the bottom part to the column sums.

    Parameters:
    N (int): The size of the grid (N x N).

    Returns:
    np.ndarray: The matrix A of size (2N x N^2) that maps the grid to the sums.
    """
    # Create an empty matrix A of the appropriate size: there are N rows and N columns, and N^2 pixels
    A = np.zeros((2 * N, N**2))

    # Fill in the top part of A for the row sums
    for i in range(N):
        for j in range(N):
            A[i, i * N + j] = 1  # Fill the row i with 1s where the pixels of row i are in the grid

    # Fill in the bottom part of A for the column sums
    for i in range(N, 2 * N):
        for j in range(N):
            A[i, j + (i - N) * N] = 1  # Fill the column i-N with 1s where the pixels of column i-N are in the grid

    return A

def convert_sums_to_matrix_Y(color_sums, N):
    """
    Convert the color_sums dictionary to a matrix Y where each column represents the color sum vector y_i.

    Parameters:
    color_sums (dict): A dictionary with the row and column sums for each color.
    N (int): The size of the grid (N x N).

    Returns:
    np.ndarray: The matrix Y containing the color sum vectors.
    """
    # The number of colors is the number of keys in the color_sums dictionary
    num_colors = len(color_sums)
    
    # Initialize the matrix Y with size 2N x number of colors (column sums on top, row sums on bottom)
    Y = np.zeros((2 * N, num_colors))

    for idx, (color, sums) in enumerate(color_sums.items()):
        # Column sums go on top, row sums on the bottom
        Y[:N, idx] = sums['col']
        Y[N:, idx] = sums['row']

    return Y

def grid_to_matrix_X(grid, colors):
    """
    Convert a nonogram grid to the matrix X, where each column corresponds to a color.

    Parameters:
    grid (np.ndarray): The grid representing the nonogram.
    colors (dict): A dictionary of colors and their corresponding labels.

    Returns:
    np.ndarray: The matrix X with size num_pixels x num_colors.
    """
    N = grid.shape[0]  # Size of the grid (N x N)
    num_pixels = N * N
    num_colors = len(colors)
    
    # Initialize the matrix X
    X = np.zeros((num_pixels, num_colors))
    
    # Create a mapping of color to column index
    color_to_index = {color: i for i, color in enumerate(colors)}
    
    # Fill in the matrix X
    for i in range(N):
        for j in range(N):
            # Get the index of the current color in the matrix X
            color_index = color_to_index[grid[j, i]]
            # Set the corresponding entry in the matrix X to 1
            X[i * N + j, color_index] = 1
    
    return X

## Main block

In [3]:
N = 5  # Grid size
colors = {'R': 'Red', 'G': 'Green', 'B': 'Blue', 'V': 'Vacant'}  # Color dictionary

true_grid, true_color_sums = initialize_nonogram(N, colors)

A = create_system_matrix(N)
Y = convert_sums_to_matrix_Y(true_color_sums, N)

X_true = grid_to_matrix_X(true_grid, colors)

print(true_grid)
print(true_color_sums)
print(A.shape)
print(Y)
print(X_true)

[['B' 'R' 'B' 'R' 'R']
 ['B' 'B' 'B' 'V' 'V']
 ['V' 'R' 'G' 'V' 'R']
 ['V' 'G' 'V' 'R' 'G']
 ['G' 'V' 'V' 'B' 'B']]
{'R': {'row': array([3, 0, 2, 1, 0]), 'col': array([0, 2, 0, 2, 2])}, 'G': {'row': array([0, 0, 1, 2, 1]), 'col': array([1, 1, 1, 0, 1])}, 'B': {'row': array([2, 3, 0, 0, 2]), 'col': array([2, 1, 2, 1, 1])}, 'V': {'row': array([0, 2, 2, 2, 2]), 'col': array([2, 1, 2, 2, 1])}}
(10, 25)
[[0. 1. 2. 2.]
 [2. 1. 1. 1.]
 [0. 1. 2. 2.]
 [2. 0. 1. 2.]
 [2. 1. 1. 1.]
 [3. 0. 2. 0.]
 [0. 0. 3. 2.]
 [2. 1. 0. 2.]
 [1. 2. 0. 2.]
 [0. 1. 2. 2.]]
[[0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


# Solving standard problem (using CVXPY)

## Functions (definitions)

In [4]:
def solve_for_X(A, Y):
    """
    Solve the convex optimization problem to find the pixel intensities X.

    Parameters:
    A (np.ndarray): The matrix A mapping the grid to the sums.
    Y (np.ndarray): The matrix Y containing the color sum vectors.

    Returns:
    np.ndarray: The matrix X containing the pixel intensities for each color channel.
    """
    num_pixels = A.shape[1]  # Number of pixels (N^2 for an N x N grid)
    num_colors = Y.shape[1]  # Number of colors (including the 'blank' channel)
    
    print(num_pixels)
    print(num_colors)
    
    # Create a variable for the intensities of each pixel for each color
    X = cp.Variable((num_pixels, num_colors))
    
    # The objective is to minimize the sum of the squares of (A * x_i - y_i) across all colors
    # objective = cp.Minimize(cp.norm(cp.hstack([A @ X[:, i] for i in range(num_colors)]) - Y, 'fro'))
    objective = cp.Minimize(cp.norm(A @ X - Y, 'fro'))
    
    # Constraints: each x_i should be between 0 and 1, and the sum across each row (pixel) should be 1
    constraints = [0 <= X, X <= 1]
    constraints += [cp.sum(X, axis=1) == 1]

    # Define and solve the problem
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS)

    return X.value

def print_matrices_side_by_side(X, X_true):
    # Stacking the two matrices horizontally
    stacked_matrix = np.hstack((X, X_true))

    # Formatting the matrix to print with 2 decimal places
    print("Matrix X (left) and Matrix X_true (right):")
    for row in stacked_matrix:
        print(" ".join(f"{val:.2f}" if isinstance(val, float) else str(val) for val in row))

## Main block

In [7]:
# solve
X = solve_for_X(A, Y)

# print
print_matrices_side_by_side(X, X_true)

25
4
Matrix X (left) and Matrix X_true (right):
0.30 0.10 0.40 0.20 0.00 0.00 1.00 0.00
0.30 0.10 0.40 0.20 0.00 0.00 1.00 0.00
0.30 0.10 0.40 0.20 0.00 0.00 0.00 1.00
0.30 0.10 0.40 0.20 0.00 0.00 0.00 1.00
0.30 0.10 0.40 0.20 0.00 1.00 0.00 0.00
0.20 0.10 0.40 0.30 1.00 0.00 0.00 0.00
0.20 0.10 0.40 0.30 0.00 0.00 1.00 0.00
0.20 0.10 0.40 0.30 1.00 0.00 0.00 0.00
0.20 0.10 0.40 0.30 0.00 1.00 0.00 0.00
0.20 0.10 0.40 0.30 0.00 0.00 0.00 1.00
0.20 0.20 0.20 0.40 0.00 0.00 1.00 0.00
0.20 0.20 0.20 0.40 0.00 0.00 1.00 0.00
0.20 0.20 0.20 0.40 0.00 1.00 0.00 0.00
0.20 0.20 0.20 0.40 0.00 0.00 0.00 1.00
0.20 0.20 0.20 0.40 0.00 0.00 0.00 1.00
0.30 0.20 0.10 0.40 1.00 0.00 0.00 0.00
0.30 0.20 0.10 0.40 0.00 0.00 0.00 1.00
0.30 0.20 0.10 0.40 0.00 0.00 0.00 1.00
0.30 0.20 0.10 0.40 1.00 0.00 0.00 0.00
0.30 0.20 0.10 0.40 0.00 0.00 1.00 0.00
0.20 0.20 0.30 0.30 1.00 0.00 0.00 0.00
0.20 0.20 0.30 0.30 0.00 0.00 0.00 1.00
0.20 0.20 0.30 0.30 1.00 0.00 0.00 0.00
0.20 0.20 0.30 0.30 0.00 1.00 0.

# Solving dual problem (using CVXPY)